In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [28]:
filenames = ["../data/newtfrecordiguess.tfr"]
raw_dataset = tf.data.TFRecordDataset(filenames)

placeholder_array = np.array([str(k) for k in range(0,128)])

for raw_record in raw_dataset:
    sample = tf.train.SequenceExample()
    sample.ParseFromString(raw_record.numpy())
    for i in range(0,len(sample.feature_lists.feature_list['audio_embedding'].feature)):
        placeholder_array = np.vstack((placeholder_array, np.array(list(sample.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0]))))

features = np.delete(placeholder_array, 0, 0).astype(float)
print(len(features))
features

16


array([[102.,  76., 166., ..., 139., 179., 222.],
       [ 72.,  81., 155., ...,  49., 136., 192.],
       [ 66.,  70., 156., ...,  22., 219., 252.],
       ...,
       [ 27.,  86., 185., ...,  98., 234., 105.],
       [ 26.,  67., 175., ...,  75., 217., 151.],
       [ 26.,  78., 168., ...,   0., 255., 102.]])

In [15]:
json_file = open('../models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights('../models/model.h5')
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print("Loaded model from disk")

predictions = np.round(model.predict(features))
predictions

Loaded model from disk


UnimplementedError:  Cast string to float is not supported
	 [[node sequential/Cast (defined at <ipython-input-15-829d419ec46f>:10) ]] [Op:__inference_predict_function_756]

Function call stack:
predict_function


In [ ]:
# Now to make the 1D heatmap visualizing the predictions:

plt.rcParams['figure.figsize']= 5,2

x = np.arange(0,len(features))
y = predictions

extent = [-1./2, 1./2, 0., 1.]
ax = plt.figure()
ax.imshow(y[np.newaxis,:], cmap='plasma', aspect='auto', extent=extent)
ax.set_yticks([])
ax.set_xlim(extent[0], extent[1])
ax.show()